In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
import xgboost
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

## XGBoost with SMOTE

In [8]:
path = r'C:\Users\Public\Documents\HRS\28.11'
beginning_of_name_EL = '\Feature matrix EL no noise'
beginning_of_name_EL_OM = '\Feature matrix EL+OM no noise'
end_of_name = 'peaks.csv'
list_of_names = [' 3 ', ' 4 ', ' 5 ']

acc = np.zeros((3,3)) # list of accuracies, shape list_of_names*3
f1 = np.zeros((3,3)) # list of f1_scores, shape list_of_names*3
f1_left_confint = np.zeros((3,3))
f1_right_confint = np.zeros((3,3))

for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    print(i)
    
    X = data.drop('target', 1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', 1) #full of OM data ~ 200 sequencies
    y1 = data1['target']
    
    target=data1['target']
    data2 = data1.iloc[:,:int((data1.shape[1]-1)/2)]
    data2 = pd.concat([data2, target], axis=1)
    
    X2 = data2.drop('target', 1) #cutted EL data to the size of OM data ~ 200 sequencies
    y2 = data2['target']

    scaler = StandardScaler()
    
    X = scaler.fit_transform(X)
    X1 = scaler.fit_transform(X1)
    X2 = scaler.fit_transform(X2)
    
    X_resampled, y_resampled = SMOTE().fit_resample(X, y)
    X1_resampled, y1_resampled = SMOTE().fit_resample(X1, y1)
    X2_resampled, y2_resampled = SMOTE().fit_resample(X2, y2)
    
    (X_train, X_test, y_train, y_test) = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled)

    (X_train_1, X_test_1, y_train_1, y_test_1) = train_test_split(X1_resampled, y1_resampled, test_size=0.3, stratify=y1_resampled)

    (X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(X2_resampled, y2_resampled, test_size=0.3, stratify=y2_resampled)
    
    model = xgboost.XGBClassifier()
    model.fit(X_train, y_train)

    model1 = xgboost.XGBClassifier()
    model1.fit(X_train_1, y_train_1)

    model2 = xgboost.XGBClassifier()
    model2.fit(X_train_2, y_train_2)

    y_pred = model.predict(X_test)
    y_pred1 = model1.predict(X_test_1)
    y_pred2 = model2.predict(X_test_2)
    
    acc[i][0]= (accuracy_score(y_test, y_pred) * 100.0)
    acc[i][1]= (accuracy_score(y_test_1, y_pred1) * 100.0)
    acc[i][2]= (accuracy_score(y_test_2, y_pred2) * 100.0)
    
    f1[i][0]= f1_score(y_test, y_pred)
    f1[i][1]= f1_score(y_test_1, y_pred1)
    f1[i][2]= f1_score(y_test_2, y_pred2)
    
    f1_scores = cross_val_score(model, X_resampled, y_resampled, scoring = 'f1',  cv = 10)
    f1_scores_1 = cross_val_score(model1, X1_resampled, y1_resampled, scoring = 'f1',  cv = 10)
    f1_scores_2 = cross_val_score(model2, X2_resampled, y2_resampled, scoring = 'f1',  cv = 10)
    
    mean = f1_scores.mean()
    mean1 = f1_scores_1.mean()
    mean2 = f1_scores_2.mean()
    
    mean_std = f1_scores.std(ddof=1)/np.sqrt(len(f1_scores))
    mean_std1 = f1_scores_1.std(ddof=1)/np.sqrt(len(f1_scores_1))
    mean_std2 = f1_scores_2.std(ddof=1)/np.sqrt(len(f1_scores_2))
    
    tconf=_tconfint_generic(mean, mean_std,len(f1_scores) - 1,0.05, 'two-sided')
    tconf_1=_tconfint_generic(mean1, mean_std1,len(f1_scores_1) - 1,0.05, 'two-sided')
    tconf_2=_tconfint_generic(mean2, mean_std2,len(f1_scores_2) - 1,0.05, 'two-sided')
    
    f1_left_confint[i][0] = tconf[0]
    f1_left_confint[i][1] = tconf_1[0]
    f1_left_confint[i][2] = tconf_2[0]
    
    f1_right_confint[i][0] = tconf[1]
    f1_right_confint[i][1] = tconf_1[1]
    f1_right_confint[i][2] = tconf_2[1]

0


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

1


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

2


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

In [9]:
f1

array([[0.90946502, 0.84210526, 0.77372263],
       [0.91546392, 0.81118881, 0.86131387],
       [0.94045175, 0.864     , 0.7751938 ]])

In [4]:
acc

array([[87.68577495, 85.82677165, 76.37795276],
       [92.14437367, 90.5511811 , 79.52755906],
       [92.99363057, 84.2519685 , 83.46456693]])

In [10]:
f1_right_confint

array([[0.89548463, 0.90014213, 0.84965194],
       [0.91038651, 0.91560932, 0.88134452],
       [0.91748802, 0.90490416, 0.88187344]])

In [11]:
f1_left_confint

array([[0.79840597, 0.74940483, 0.66692828],
       [0.82781414, 0.714409  , 0.68475494],
       [0.84707083, 0.73897528, 0.70799683]])

In [12]:
acc = np.zeros((3,3)) # list of accuracies, shape list_of_names*3
f1 = np.zeros((3,3)) # list of f1_scores, shape list_of_names*3
f1_left_confint = np.zeros((3,3))
f1_right_confint = np.zeros((3,3))

for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    print(i)
    
    X = data.drop('target', 1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', 1) #full of OM data ~ 200 sequencies
    y1 = data1['target']
    
    target=data1['target']
    data2 = data1.iloc[:,:int((data1.shape[1]-1)/2)]
    data2 = pd.concat([data2, target], axis=1)
    
    X2 = data2.drop('target', 1) #cutted EL data to the size of OM data ~ 200 sequencies
    y2 = data2['target']
    
    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.3, stratify=y)
    (X_train_1, X_test_1, y_train_1, y_test_1) = train_test_split(X1, y1, test_size=0.3, stratify=y1)
    (X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(X2, y2, test_size=0.3, stratify=y2)
    
    encoder = StandardScaler()
    X_train_scaled = encoder.fit_transform(X_train)
    X_test = encoder.fit_transform(X_test)
    X_train, y_train = SMOTE().fit_resample(X_train_scaled, y_train)
    
    X_train_scaled_1 = encoder.fit_transform(X_train_1)
    X_test_1 = encoder.fit_transform(X_test_1)
    X_train_1, y_train_1 = SMOTE().fit_resample(X_train_scaled_1, y_train_1)
    
    X_train_scaled_2 = encoder.fit_transform(X_train_2)
    X_test_2 = encoder.fit_transform(X_test_2)
    X_train_2, y_train_2 = SMOTE().fit_resample(X_train_scaled_2, y_train_2)
    
    model = xgboost.XGBClassifier()
    model.fit(X_train, y_train)

    model1 = xgboost.XGBClassifier()
    model1.fit(X_train_1, y_train_1)

    model2 = xgboost.XGBClassifier()
    model2.fit(X_train_2, y_train_2)

    y_pred = model.predict(X_test)
    y_pred1 = model1.predict(X_test_1)
    y_pred2 = model2.predict(X_test_2)
    
    acc[i][0]= (accuracy_score(y_test, y_pred) * 100.0)
    acc[i][1]= (accuracy_score(y_test_1, y_pred1) * 100.0)
    acc[i][2]= (accuracy_score(y_test_2, y_pred2) * 100.0)
    
    f1[i][0]= f1_score(y_test, y_pred)
    f1[i][1]= f1_score(y_test_1, y_pred1)
    f1[i][2]= f1_score(y_test_2, y_pred2)
    
    f1_scores = cross_val_score(model, X, y, scoring = 'f1',  cv = 10)
    f1_scores_1 = cross_val_score(model1, X1, y1, scoring = 'f1',  cv = 10)
    f1_scores_2 = cross_val_score(model2, X2, y2, scoring = 'f1',  cv = 10)
    
    mean = f1_scores.mean()
    mean1 = f1_scores_1.mean()
    mean2 = f1_scores_2.mean()
    
    mean_std = f1_scores.std(ddof=1)/np.sqrt(len(f1_scores))
    mean_std1 = f1_scores_1.std(ddof=1)/np.sqrt(len(f1_scores_1))
    mean_std2 = f1_scores_2.std(ddof=1)/np.sqrt(len(f1_scores_2))
    
    tconf=_tconfint_generic(mean, mean_std,len(f1_scores) - 1,0.05, 'two-sided')
    tconf_1=_tconfint_generic(mean1, mean_std1,len(f1_scores_1) - 1,0.05, 'two-sided')
    tconf_2=_tconfint_generic(mean2, mean_std2,len(f1_scores_2) - 1,0.05, 'two-sided')
    
    f1_left_confint[i][0] = tconf[0]
    f1_left_confint[i][1] = tconf_1[0]
    f1_left_confint[i][2] = tconf_2[0]
    
    f1_right_confint[i][0] = tconf[1]
    f1_right_confint[i][1] = tconf_1[1]
    f1_right_confint[i][2] = tconf_2[1]

0


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

1


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

2


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

In [13]:
f1

array([[0.18181818, 0.37209302, 0.34146341],
       [0.21538462, 0.21621622, 0.17647059],
       [0.13793103, 0.35897436, 0.33333333]])

In [14]:
f1_left_confint

array([[ 0.        , -0.0215154 , -0.03029709],
       [ 0.        ,  0.00217778, -0.02157627],
       [ 0.        , -0.02655295,  0.        ]])

In [15]:
f1_right_confint

array([[0.        , 0.26818207, 0.1414082 ],
       [0.        , 0.2756    , 0.2549096 ],
       [0.        , 0.12814025, 0.        ]])

## XGBoost with only StandartScaler

In [16]:
acc = np.zeros((3,3)) # list of accuracies, shape list_of_names*3
f1 = np.zeros((3,3)) # list of f1_scores, shape list_of_names*3
f1_left_confint = np.zeros((3,3))
f1_right_confint = np.zeros((3,3))

for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    print(i)
    
    X = data.drop('target', 1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', 1) #full of OM data ~ 200 sequencies
    y1 = data1['target']
    
    target=data1['target']
    data2 = data1.iloc[:,:int((data1.shape[1]-1)/2)]
    data2 = pd.concat([data2, target], axis=1)
    
    X2 = data2.drop('target', 1) #cutted EL data to the size of OM data ~ 200 sequencies
    y2 = data2['target']
    
    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.3, stratify=y)
    (X_train_1, X_test_1, y_train_1, y_test_1) = train_test_split(X1, y1, test_size=0.3, stratify=y1)
    (X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(X2, y2, test_size=0.3, stratify=y2)
    
    encoder = StandardScaler()
    X_train = encoder.fit_transform(X_train)
    X_test = encoder.fit_transform(X_test)
    
    X_train_1 = encoder.fit_transform(X_train_1)
    X_test_1 = encoder.fit_transform(X_test_1)
    
    X_train_2 = encoder.fit_transform(X_train_2)
    X_test_2 = encoder.fit_transform(X_test_2)
    
    model = xgboost.XGBClassifier()
    model.fit(X_train, y_train)

    model1 = xgboost.XGBClassifier()
    model1.fit(X_train_1, y_train_1)

    model2 = xgboost.XGBClassifier()
    model2.fit(X_train_2, y_train_2)

    y_pred = model.predict(X_test)
    y_pred1 = model1.predict(X_test_1)
    y_pred2 = model2.predict(X_test_2)
    
    acc[i][0]= (accuracy_score(y_test, y_pred) * 100.0)
    acc[i][1]= (accuracy_score(y_test_1, y_pred1) * 100.0)
    acc[i][2]= (accuracy_score(y_test_2, y_pred2) * 100.0)
    
    f1[i][0]= f1_score(y_test, y_pred)
    f1[i][1]= f1_score(y_test_1, y_pred1)
    f1[i][2]= f1_score(y_test_2, y_pred2)
    
    f1_scores = cross_val_score(model, X, y, scoring = 'f1',  cv = 10)
    f1_scores_1 = cross_val_score(model1, X1, y1, scoring = 'f1',  cv = 10)
    f1_scores_2 = cross_val_score(model2, X2, y2, scoring = 'f1',  cv = 10)
    
    mean = f1_scores.mean()
    mean1 = f1_scores_1.mean()
    mean2 = f1_scores_2.mean()
    
    mean_std = f1_scores.std(ddof=1)/np.sqrt(len(f1_scores))
    mean_std1 = f1_scores_1.std(ddof=1)/np.sqrt(len(f1_scores_1))
    mean_std2 = f1_scores_2.std(ddof=1)/np.sqrt(len(f1_scores_2))
    
    tconf=_tconfint_generic(mean, mean_std,len(f1_scores) - 1,0.05, 'two-sided')
    tconf_1=_tconfint_generic(mean1, mean_std1,len(f1_scores_1) - 1,0.05, 'two-sided')
    tconf_2=_tconfint_generic(mean2, mean_std2,len(f1_scores_2) - 1,0.05, 'two-sided')
    
    f1_left_confint[i][0] = tconf[0]
    f1_left_confint[i][1] = tconf_1[0]
    f1_left_confint[i][2] = tconf_2[0]
    
    f1_right_confint[i][0] = tconf[1]
    f1_right_confint[i][1] = tconf_1[1]
    f1_right_confint[i][2] = tconf_2[1]

0


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

1


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

2


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

In [17]:
f1

array([[0.        , 0.27272727, 0.23076923],
       [0.        , 0.38095238, 0.        ],
       [0.        , 0.11111111, 0.        ]])

In [18]:
acc

array([[92.06349206, 79.48717949, 74.35897436],
       [92.85714286, 83.33333333, 70.51282051],
       [93.25396825, 79.48717949, 75.64102564]])

## UnderSampling

In [19]:
from imblearn.under_sampling import RandomUnderSampler

In [20]:
acc = np.zeros((3,3)) # list of accuracies, shape list_of_names*3
f1 = np.zeros((3,3)) # list of f1_scores, shape list_of_names*3
f1_left_confint = np.zeros((3,3))
f1_right_confint = np.zeros((3,3))

for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    print(i)
    
    X = data.drop('target', 1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', 1) #full of OM data ~ 200 sequencies
    y1 = data1['target']
    
    target=data1['target']
    data2 = data1.iloc[:,:int((data1.shape[1]-1)/2)]
    data2 = pd.concat([data2, target], axis=1)
    
    X2 = data2.drop('target', 1) #cutted EL data to the size of OM data ~ 200 sequencies
    y2 = data2['target']

    scaler = StandardScaler()
    
    X = scaler.fit_transform(X)
    X1 = scaler.fit_transform(X1)
    X2 = scaler.fit_transform(X2)
    
    X_resampled, y_resampled = RandomUnderSampler().fit_resample(X, y)
    X1_resampled, y1_resampled = RandomUnderSampler().fit_resample(X1, y1)
    X2_resampled, y2_resampled = RandomUnderSampler().fit_resample(X2, y2)
    
    (X_train, X_test, y_train, y_test) = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled)

    (X_train_1, X_test_1, y_train_1, y_test_1) = train_test_split(X1_resampled, y1_resampled, test_size=0.3, stratify=y1_resampled)

    (X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(X2_resampled, y2_resampled, test_size=0.3, stratify=y2_resampled)
    
    model = xgboost.XGBClassifier()
    model.fit(X_train, y_train)

    model1 = xgboost.XGBClassifier()
    model1.fit(X_train_1, y_train_1)

    model2 = xgboost.XGBClassifier()
    model2.fit(X_train_2, y_train_2)

    y_pred = model.predict(X_test)
    y_pred1 = model1.predict(X_test_1)
    y_pred2 = model2.predict(X_test_2)
    
    acc[i][0]= (accuracy_score(y_test, y_pred) * 100.0)
    acc[i][1]= (accuracy_score(y_test_1, y_pred1) * 100.0)
    acc[i][2]= (accuracy_score(y_test_2, y_pred2) * 100.0)
    
    f1[i][0]= f1_score(y_test, y_pred)
    f1[i][1]= f1_score(y_test_1, y_pred1)
    f1[i][2]= f1_score(y_test_2, y_pred2)
    
    f1_scores = cross_val_score(model, X_resampled, y_resampled, scoring = 'f1',  cv = 10)
    f1_scores_1 = cross_val_score(model1, X1_resampled, y1_resampled, scoring = 'f1',  cv = 10)
    f1_scores_2 = cross_val_score(model2, X2_resampled, y2_resampled, scoring = 'f1',  cv = 10)
    
    mean = f1_scores.mean()
    mean1 = f1_scores_1.mean()
    mean2 = f1_scores_2.mean()
    
    mean_std = f1_scores.std(ddof=1)/np.sqrt(len(f1_scores))
    mean_std1 = f1_scores_1.std(ddof=1)/np.sqrt(len(f1_scores_1))
    mean_std2 = f1_scores_2.std(ddof=1)/np.sqrt(len(f1_scores_2))
    
    tconf=_tconfint_generic(mean, mean_std,len(f1_scores) - 1,0.05, 'two-sided')
    tconf_1=_tconfint_generic(mean1, mean_std1,len(f1_scores_1) - 1,0.05, 'two-sided')
    tconf_2=_tconfint_generic(mean2, mean_std2,len(f1_scores_2) - 1,0.05, 'two-sided')
    
    f1_left_confint[i][0] = tconf[0]
    f1_left_confint[i][1] = tconf_1[0]
    f1_left_confint[i][2] = tconf_2[0]
    
    f1_right_confint[i][0] = tconf[1]
    f1_right_confint[i][1] = tconf_1[1]
    f1_right_confint[i][2] = tconf_2[1]

0


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

1


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

2


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

In [21]:
f1

array([[0.57894737, 0.57142857, 0.51612903],
       [0.57142857, 0.46666667, 0.46666667],
       [0.56410256, 0.60606061, 0.5       ]])

In [22]:
f1_left_confint

array([[0.34248131, 0.2984295 , 0.24437299],
       [0.30671193, 0.29663155, 0.15533453],
       [0.35506288, 0.22830592, 0.18147577]])

In [23]:
f1_right_confint

array([[0.69265911, 0.56804292, 0.5785996 ],
       [0.59418606, 0.72078548, 0.52522158],
       [0.58235748, 0.58169408, 0.55190085]])

In [24]:
acc = np.zeros((3,3)) # list of accuracies, shape list_of_names*3
f1 = np.zeros((3,3)) # list of f1_scores, shape list_of_names*3
f1_left_confint = np.zeros((3,3))
f1_right_confint = np.zeros((3,3))

for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    print(i)
    
    X = data.drop('target', 1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', 1) #full of OM data ~ 200 sequencies
    y1 = data1['target']
    
    target=data1['target']
    data2 = data1.iloc[:,:int((data1.shape[1]-1)/2)]
    data2 = pd.concat([data2, target], axis=1)
    
    X2 = data2.drop('target', 1) #cutted EL data to the size of OM data ~ 200 sequencies
    y2 = data2['target']
    
    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.3, stratify=y)
    (X_train_1, X_test_1, y_train_1, y_test_1) = train_test_split(X1, y1, test_size=0.3, stratify=y1)
    (X_train_2, X_test_2, y_train_2, y_test_2) = train_test_split(X2, y2, test_size=0.3, stratify=y2)
    
    encoder = StandardScaler()
    X_train_scaled = encoder.fit_transform(X_train)
    X_test = encoder.fit_transform(X_test)
    X_train, y_train = RandomUnderSampler().fit_resample(X_train_scaled, y_train)
    
    X_train_scaled_1 = encoder.fit_transform(X_train_1)
    X_test_1 = encoder.fit_transform(X_test_1)
    X_train_1, y_train_1 = RandomUnderSampler().fit_resample(X_train_scaled_1, y_train_1)
    
    X_train_scaled_2 = encoder.fit_transform(X_train_2)
    X_test_2 = encoder.fit_transform(X_test_2)
    X_train_2, y_train_2 = RandomUnderSampler().fit_resample(X_train_scaled_2, y_train_2)
    
    model = xgboost.XGBClassifier()
    model.fit(X_train, y_train)

    model1 = xgboost.XGBClassifier()
    model1.fit(X_train_1, y_train_1)

    model2 = xgboost.XGBClassifier()
    model2.fit(X_train_2, y_train_2)

    y_pred = model.predict(X_test)
    y_pred1 = model1.predict(X_test_1)
    y_pred2 = model2.predict(X_test_2)
    
    acc[i][0]= (accuracy_score(y_test, y_pred) * 100.0)
    acc[i][1]= (accuracy_score(y_test_1, y_pred1) * 100.0)
    acc[i][2]= (accuracy_score(y_test_2, y_pred2) * 100.0)
    
    f1[i][0]= f1_score(y_test, y_pred)
    f1[i][1]= f1_score(y_test_1, y_pred1)
    f1[i][2]= f1_score(y_test_2, y_pred2)
    
    f1_scores = cross_val_score(model, X, y, scoring = 'f1',  cv = 10)
    f1_scores_1 = cross_val_score(model1, X1, y1, scoring = 'f1',  cv = 10)
    f1_scores_2 = cross_val_score(model2, X2, y2, scoring = 'f1',  cv = 10)
    
    mean = f1_scores.mean()
    mean1 = f1_scores_1.mean()
    mean2 = f1_scores_2.mean()
    
    mean_std = f1_scores.std(ddof=1)/np.sqrt(len(f1_scores))
    mean_std1 = f1_scores_1.std(ddof=1)/np.sqrt(len(f1_scores_1))
    mean_std2 = f1_scores_2.std(ddof=1)/np.sqrt(len(f1_scores_2))
    
    tconf=_tconfint_generic(mean, mean_std,len(f1_scores) - 1,0.05, 'two-sided')
    tconf_1=_tconfint_generic(mean1, mean_std1,len(f1_scores_1) - 1,0.05, 'two-sided')
    tconf_2=_tconfint_generic(mean2, mean_std2,len(f1_scores_2) - 1,0.05, 'two-sided')
    
    f1_left_confint[i][0] = tconf[0]
    f1_left_confint[i][1] = tconf_1[0]
    f1_left_confint[i][2] = tconf_2[0]
    
    f1_right_confint[i][0] = tconf[1]
    f1_right_confint[i][1] = tconf_1[1]
    f1_right_confint[i][2] = tconf_2[1]

0


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

1


C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetri

2


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype in

In [25]:
acc

array([[53.96825397, 46.15384615, 64.1025641 ],
       [54.76190476, 56.41025641, 56.41025641],
       [54.36507937, 52.56410256, 51.28205128]])

In [26]:
f1

array([[0.09375   , 0.32258065, 0.33333333],
       [0.18571429, 0.39285714, 0.37037037],
       [0.09448819, 0.32727273, 0.36666667]])